In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy import sparse
import re
import os

In [52]:
fp_tutorial_path = "/cluster/work/bewi/members/rquiles/zeroshot_amr/tutorial/data/drug_fingerprints_Mol_selfies.csv"
fp_path = "/cluster/work/bewi/members/rquiles/zeroshot_amr/data/drug_fingerprints_Mol_selfies.csv"

df_tutorial = pd.read_csv(fp_tutorial_path)
df = pd.read_csv(fp_path)

In [54]:
df.head()

,drug,morgan_512_fp,morgan_1024_fp,MACCS_fp
0,Talc,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000100000000010000000000000000000000010...
1,Bortezomib,0100000000000000000000000000010001000000101000...,0100000000000000000000000000000001000000001000...,0000000000000000001000000000000000000000000110...
2,Ixazomib,0100000000001000000000000000010001000000010000...,0100000000000000000000000000000001000000010000...,0000000000000000001000000000000000000000000110...
3,Ixazomib citrate,1100000000001000000000000000000001001000010000...,1100000000000000000000000000000001001000010000...,0000000000000000001000000000000000000000000010...
4,Lactate (calcium),0100000000000000000000000000000001000000000000...,0100000000000000000000000000000001000000000000...,0000000000100000000000000000000000000000000010...


In [46]:
df.columns

Index(['Unnamed: 0', 'drug', 'targets', 'moa-broad', 'moa-fine',
       'human-approved', 'clinical-trials', 'gpt-notes-approval',
       'canonical_smiles', 'pubchem_cid', 'chemberta', 'morgan_fp',
       'morgan_512_fp', 'morgan_1024_fp', 'MACCS_fp'],
      dtype='object')

In [47]:
# Drop unwanted columns
keep = ["drug", "morgan_512_fp", "morgan_1024_fp", "MACCS_fp"]   # example
popped = []

for col in df.columns.tolist():
    if col not in keep:
        popped.append(col)

df = df.drop(popped, axis=1)

In [48]:
# Map the kept columns to a desired format
def unstring(s):
    return "".join(s.strip("[]").split())

for col in keep:
    if col != "drug":
        df[col] = df[col].map(unstring)

In [50]:
df

,drug,morgan_512_fp,morgan_1024_fp,MACCS_fp
0,Talc,0000000000000000000000000000000000000000000000...,0000000000000000000000000000000000000000000000...,0000000000100000000010000000000000000000000010...
1,Bortezomib,0100000000000000000000000000010001000000101000...,0100000000000000000000000000000001000000001000...,0000000000000000001000000000000000000000000110...
2,Ixazomib,0100000000001000000000000000010001000000010000...,0100000000000000000000000000000001000000010000...,0000000000000000001000000000000000000000000110...
3,Ixazomib citrate,1100000000001000000000000000000001001000010000...,1100000000000000000000000000000001001000010000...,0000000000000000001000000000000000000000000010...
4,Lactate (calcium),0100000000000000000000000000000001000000000000...,0100000000000000000000000000000001000000000000...,0000000000100000000000000000000000000000000010...
...,...,...,...,...
375,Quinidine (15% dihydroquinidine),1100000000000000000000000000000001000000000000...,0100000000000000000000000000000001000000000000...,0000000000000000000000000000000000100000000000...
376,Canagliflozin (hemihydrate),0000010000000101000000000000000001000000000000...,0000000000000101000000000000000001000000000000...,0000000000000000000000000000000000001000001000...
377,Osimertinib (mesylate),1000000000000000000000000000000001000100000101...,0000000000000000000000000000000001000000000001...,0000000000000000000000000100000000100001100000...
378,γ-Oryzanol,1100100010100000000000100100000101001110000110...,0100100010000000000000100100000101001000000000...,0000000000000000000100100000000000000000000000...


In [51]:
# Export again
df.to_csv(fp_path, index=False)